In [ ]:
%run -i utils.py

In [ ]:
# Set file paths and load files
file_path = Path(
    'eeg-data/Stephanie/Rew_601_rest/Rew_601_rest_bb_epoch.set')
mat_reject = Path(
    'eeg-data/Stephanie/Rew_601_rest/Rew_601_rest_reject_rmm.mat')
mat_stage = Path(
    'eeg-data/Stephanie/Rew_601_rest/Rew_601_rest_stages.mat')

In [ ]:
files = load_subject_dir(file_path, mat_reject, mat_stage)
epochs = files['epochs']
try:
    rejects = files['rejects']
except:
    pass

try:
    stages = files['stages']
except:
    pass